In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 1. generalized online tensor factorization (GOTF)

### 1.1 data preparation

In [15]:
from GOCPT import datasets, simulation, otf, gotf
import numpy as np

# load Columbia FACE 3D tensor
X = datasets.FACE_3D()

# simulate one mode growth (typically the last mode)
# use 30% as preparation and increase 3 slices at each step
[X_0, X_inc_ls] = simulation.growth_1d(X, prep=0.3, inc=1)


        ---------- new OTF setting ------------
        base tensor size: (112, 92, 120),
        new tensor increment size: (112, 92, 1),
        tensor will be updated 279 times.
        


### 1.2 our GOCPT and GOCPTE model

In [16]:
# initialize model
model = gotf.GOCPTE(X_0, R=5)
# model = gotf.GOCPT(X_0, R=5)

# update model with new slices, new_R can change during for-loop
for increments in X_inc_ls[:10]:
    new_R = np.random.choice([5,6,7,8])
    model.update(increments, new_R=new_R)

Initial PoF Metric: 0.7471367150994004
This model does not need to prepare aux!

1-th update Success! PoF: 0.747, run_time: 0.005804s
2-th update Success! PoF: 0.7473, run_time: 0.01592s
3-th update Success! PoF: 0.7474, run_time: 0.005861s
4-th update Success! PoF: 0.7479, run_time: 0.005744s
5-th update Success! PoF: 0.7482, run_time: 0.01652s
6-th update Success! PoF: 0.748, run_time: 0.005734s
7-th update Success! PoF: 0.7479, run_time: 0.005741s
8-th update Success! PoF: 0.748, run_time: 0.005725s
9-th update Success! PoF: 0.7478, run_time: 0.01681s
10-th update Success! PoF: 0.7473, run_time: 0.005657s


### 1.3 baseline models

In [13]:
# baseline models for online tensor factorization (OTF)

baseline = otf.MAST(X_0, R=5)
# baseline = otf.OnlineCPD(X_0, R=5)
# baseline = otf.SDT(X_0, R=5)
# baseline = otf.RLST(X_0, R=5)
# baseline = otf.CPStream(X_0, R=5)

# update model with new slices
for increments in X_inc_ls[:10]:
    baseline.update(increments)

Initial PoF Metric: 0.7470695791372648
This model does not need to prepare aux!

1-th update, PoF: 0.7471, run_time: 0.1365s
2-th update, PoF: 0.7473, run_time: 0.1039s
3-th update, PoF: 0.7476, run_time: 0.1382s
4-th update, PoF: 0.748, run_time: 0.1401s
5-th update, PoF: 0.7483, run_time: 0.1395s
6-th update, PoF: 0.748, run_time: 0.1405s
7-th update, PoF: 0.7478, run_time: 0.1409s
8-th update, PoF: 0.7479, run_time: 0.0802s
9-th update, PoF: 0.7477, run_time: 0.1415s
10-th update, PoF: 0.7473, run_time: 0.1463s


# 2. generalized online tensor completion (GOTC)

### 2.1 data preparation

In [27]:
from GOCPT import datasets, simulation, otc, gotc
import numpy as np

# load synthetic data
[masked_X, mask] = datasets.syn_data(R=5, size=(5, 10, 15, 50), dist='unif', sparsity=0.95)

# simulate one mode growth (typically the last mode)
# use 30% as preparation and increase 3 slices at each step
[[X_0, mask_0], [X_inc_ls, mask_inc_ls]] = simulation.growth_1d([masked_X, mask], prep=0.3, inc=3)


        ---------- new OTC setting ------------
        base tensor size: (5, 10, 15, 15),
        new tensor increment size: (5, 10, 15, 3),
        tensor will be updated 12 times.
        


### 2.2 Our GOCPT and GOCPTE model

In [28]:
# model initialization
# model = gotc.GOCPTE([X_0, mask_0], R=5)
model = gotc.GOCPT([X_0, mask_0], R=5)

# update model with new slices, new_R can change during for-loop
for increments in zip(X_inc_ls, mask_inc_ls):
    # simulate value update， missing filling，change of rank
    new_value_update = simulation.value_update(model.X, model.mask, percent=10, amp=0.05)
    new_missing_fill = simulation.missing_fill(model.X, model.mask, percent=10, factors=model.factors)
    new_R = np.random.choice([5,6,7,8])
    model.update(increments, new_R=new_R, value_update=new_value_update, miss_fill=new_missing_fill)

Initial PoF Metric: 0.9390867904915358
This model does not need to prepare aux!

number of newly updated entries: 10 / 572
number of newly filled entries: 10 / 10678
1-th update Success! PoF: 0.9323, run_time: 0.009394s

number of newly updated entries: 10 / 697
number of newly filled entries: 10 / 12803
2-th update Success! PoF: 0.9266, run_time: 0.009832s

number of newly updated entries: 10 / 844
number of newly filled entries: 10 / 14906
3-th update Success! PoF: 0.9235, run_time: 0.01003s

number of newly updated entries: 10 / 971
number of newly filled entries: 10 / 17029
4-th update Success! PoF: 0.9236, run_time: 0.01317s

number of newly updated entries: 10 / 1086
number of newly filled entries: 10 / 19164
5-th update Success! PoF: 0.9191, run_time: 0.02472s

number of newly updated entries: 10 / 1217
number of newly filled entries: 10 / 21283
6-th update Success! PoF: 0.9219, run_time: 0.02892s

number of newly updated entries: 10 / 1342
number of newly filled entries: 10 / 2

### 2.3 Our GOCPT and GOCPTE model

In [29]:
model = otc.OnlineSGD([X_0, mask_0], R=5)

for increments in zip(X_inc_ls, mask_inc_ls):
    model.update(increments)

Initial PoF Metric: 0.9112019640840954
This model does not need to prepare aux!



ValueError: Size of label 'd' for operand 3 (3) does not match previous terms (15).